In [1]:
import torch
import torch.nn as nn
import time
import argparse
import networkx as nx
import os

from st_graph_generation import detect_with_networkx
from common_utils.stg_utils import load_video_params
from common_utils.process import graph_to_feature_vector, adj_to_normalized_tensor
from embedding_training.transformer_classifier import Net
from anomaly_generation.graph_corruption import Corruptor

VIDEOS_FOLDER = "../videos/"
GRAPH_FOLDER = "../graphs/"

In [2]:
# to run only once to convert all the videos in VIDEOS_FOLDER into graphs
with torch.no_grad():
    videoname_to_graph_seq = detect_with_networkx.videos_to_graphs(source=VIDEOS_FOLDER, dest=GRAPH_FOLDER, save_graphs=True)

YOLOR 🚀 2022-11-24 torch 1.12.1 CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


Model Summary: 306 layers, 36905341 parameters, 6652669 gradients



video 1/4 (14835/64797) adoc_part1.avi

/Users/niccolomorabito/Library/Python/3.9/lib/python/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


video 1/4 (15002/64797) adoc_part1.avi

IndexError: index 8 is out of bounds for axis 1 with size 6

In [ ]:
'''
for videoname, graph_seq in videoname_to_graph_seq.items():
    print(f"Saving graphs of {videoname} ({len(videoname_to_graph_seq[videoname])} frames)")
    videoname_folder = os.path.join(GRAPH_FOLDER, videoname)
    os.mkdir(videoname_folder)
    for i, graph in enumerate(graph_seq):
        graph_path = os.path.join(videoname_folder, f"{i}.gpickle")
        nx.write_gpickle(graph,graph_path)
'''

'\nfor videoname, graph_seq in videoname_to_graph_seq.items():\n    print(f"Saving graphs of {videoname} ({len(videoname_to_graph_seq[videoname])} frames)")\n    videoname_folder = os.path.join(GRAPH_FOLDER, videoname)\n    os.mkdir(videoname_folder)\n    for i, graph in enumerate(graph_seq):\n        graph_path = os.path.join(videoname_folder, f"{i}.gpickle")\n        nx.write_gpickle(graph,graph_path)\n'

In [2]:
for videoname in os.listdir(GRAPH_FOLDER):
    if videoname.startswith('.'):
        continue
    videoname_folder = os.path.join(GRAPH_FOLDER, videoname)
    print(videoname_folder)
    # if in the list comprehension is to get only files for frames
    graph_paths = [os.path.join(videoname_folder, p) for p in os.listdir(videoname_folder) if p[0].isdigit()]
    graph_seq = [nx.read_gpickle(graph_path) for graph_path in graph_paths]

    # CORRUPTION
    video_params = load_video_params(videoname_folder)
    w, h = video_params["width"], video_params["height"]
    #TODO so far, is_stg must be true because only the distance is implemented on the edge as 'weight'
    corruptor = Corruptor(frame_width=w, frame_height=h, is_stg=True)

    corrupted_graph_seq = [corruptor.corrupt_graph(graph) for graph in graph_seq]
    
print(len(graph_seq))
print(len(corrupted_graph_seq))

../graphs/adoc_part1.avi
15001
15001


In [ ]:
features = graph_to_feature_vector(graph)
features.shape

adj = nx.adjacency_matrix(graph)
adj = adj_to_normalized_tensor(adj)
adj.shape

/var/folders/9c/3x13fm0x7034gf06x85_2t180000gn/T/ipykernel_40657/2767233667.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = adjacency_matrix(graph)


torch.Size([1, 22, 22])

In [ ]:

model = Net(features_size=features.shape[2], embedding_size=8) #TODO decide a value for embedding vector (MUST BE EVEN) a low number is probably enough since most of the features are one-hot encoded ones
model.forward(features, adj)

torch.Size([1, 22, 85])
torch.Size([1, 22, 22])
torch.Size([1, 22, 8])
torch.Size([1, 8])
beginning with transformer
torch.Size([1, 1, 8])
torch.Size([1, 1, 8])
torch.Size([1, 1, 1])
torch.Size([1, 1, 1])


tensor([[[0.43121]]], grad_fn=<SigmoidBackward0>)

In [ ]:
''' TRAINING PART [WIP]'''

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 50
model = Net(
    features_size=features.shape[2], # i.e. 85
    embedding_size=8 #TODO decide a value for embedding vector (MUST BE EVEN and dividible by nhead) a low number is probably enough since most of the features are one-hot encoded ones
).to(device)

criterion = nn.CrossEntropyLoss()
lr = 1e-4
optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=lr
)
torch.manual_seed(0)

print("starting")
for epoch in range(epochs):
    print(f"{epoch=}")
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for batch in iter(train_iter):
        predictions = model(batch.text.to(device)) #TODO change
        labels = batch.label.to(device) - 1

        loss = criterion(predictions, labels)

        correct = predictions.argmax(axis=1) == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()

    with torch.no_grad():
        test_epoch_loss = 0
        test_epoch_correct = 0
        test_epoch_count = 0

        for batch in iter(test_iter):
            predictions = model(batch.text.to(device))
            labels = batch.label.to(device) - 1
            test_loss = criterion(predictions, labels)

            correct = predictions.argmax(axis=1) == labels
            acc = correct.sum().item() / correct.size(0)

            test_epoch_correct += correct.sum().item()
            test_epoch_count += correct.size(0)
            test_epoch_loss += loss.item()

    print(f"{epoch_loss=}")
    print(f"epoch accuracy: {epoch_correct / epoch_count}")
    print(f"{test_epoch_loss=}")
    print(f"test epoch accuracy: {test_epoch_correct / test_epoch_count}")

starting
epoch=0


NameError: name 'train_iter' is not defined

# CORRUPTION
(see if it must be done during the process or it can be done as follows after all)

HOW CORRUPTUION CODE SHOULD BE CALLED

```
for videoclip:
    corruptor = Corruptor(..., ...)
    normal_sequence = list()
    corrupted_sequence = list()

    for frame in clip:
        graph = //graph initialization
        corrupted_graph = corruptor.corrupt_graph(graph)
        normal_sequence.append(graph)
        corrupted_sequence.append(corrupted_graph)
        // do we need object tracking etc.???
```